# Объединение аннотаций для дублирующихся файлов в REALEC

В корпусе REALEC много студенческих учебных текстов на английском языке, и иногда случается, что один и тот же текст заносится в корпус дважды и аннотируется дважды. В рамках этого задания предлагается автоматизировать слияние различающихся аннотаций для таких текстов в один файл и вывести таблицу конфликтов для аннотаций, которые перекрываются.

### Подготовка среды

Много полезных функций, все из которых подготавливают рабочее пространство для выполнения задания. Выполняются где-то около 15 минут, так что можно заварить кофе и посмотреть пару видео на ютубе.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Скачиваем специально написанную мной библиотеку для обработки используемого в REALEC типа нотации ошибок в файлах `.ann`, основанного на `brat`:

In [ ]:
!git clone --quiet https://github.com/isikus/corpora-manipulation
!cp ./corpora-manipulation/* .

In [ ]:
import re
import os
import json
import shutil
import pandas as pd

from tensorflow import keras
from collections import OrderedDict
from nltk.tokenize import sent_tokenize
from realec_brat_to_patch_list import ann_to_patchlist
from tqdm.auto import tqdm

tqdm.pandas()

Скачиваем актуальный дамп корпуса напрямую из REALEC. По опыту, это занимает либо 15 секунд, либо 6 минут – без третьего варианта. Так что как повезёт:

In [ ]:
dataset = keras.utils.get_file(
    fname="data.tar.gz", 
    origin="http://realec.org/ajax.cgi?action=downloadCollection&collection=%2F&protocol=1", 
    extract=True)

77340672/Unknown - 21s 0us/step

In [ ]:
shutil.copytree("/root/.keras/datasets/data","./data")

'./data'

In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Обработка корпуса

В REALEC существует множество типов ошибок, которые структурированы в иерархическое многоуровневое дерево. В последующей скрытой ячейке мы записываем их, чтобы убрать из рассматриваемых текстов аннотации частей речи (их добавим позднее) и невалидные ошибки:

In [ ]:
#@title Список кодов типов ошибок
error_typify = {
  "Punctuation": "Punctuation",
  "Spelling": "Spelling",
  "Capitalisation": "Spelling",
  "Grammar": "Word-level grammar",
  "Determiners": "Word-level grammar",
  "Articles": "Word-level grammar",
  "Quantifiers": "Word-level grammar",
  "Verbs": "Word-level grammar",
  "Tense": "Word-level grammar",
  "Tense_choice": "Word-level grammar",
  "Tense_form": "Word-level grammar",
  "Voice": "Word-level grammar",
  "Modals": "Word-level grammar",
  "Verb_pattern": "Word-level grammar",
  "Intransitive": "Word-level grammar",
  "Transitive": "Word-level grammar",
  "Reflexive_verb": "Word-level grammar",
  "Presentation": "Word-level grammar",
  "Ambitransitive": "Word-level grammar",
  "Two_in_a_row": "Word-level grammar",
  "Verb_Inf": "Word-level grammar",
  "Verb_Gerund": "Word-level grammar",
  "Verb_Inf_Gerund": "Word-level grammar",
  "Verb_Bare_Inf": "Word-level grammar",
  "Verb_object_bare": "Word-level grammar",
  "Restoration_alter": "Word-level grammar",
  "Verb_part": "Word-level grammar",
  "Get_part": "Word-level grammar",
  "Complex_obj": "Word-level grammar",
  "Verbal_idiom": "Word-level grammar",
  "Prepositional_verb": "Word-level grammar",
  "Dative": "Word-level grammar",
  "Followed_by_a_clause": "Word-level grammar",
  "that_clause": "Word-level grammar",
  "if_whether_clause": "Word-level grammar",
  "that_subj_clause": "Word-level grammar",
  "it_conj_clause": "Word-level grammar",
  "Participial_constr": "Word-level grammar",
  "Infinitive_constr": "Word-level grammar",
  "Gerund_phrase": "Word-level grammar",
  "Nouns": "Word-level grammar",
  "Countable_uncountable": "Word-level grammar",
  "Prepositional_noun": "Word-level grammar",
  "Possessive": "Word-level grammar",
  "Noun_attribute": "Word-level grammar",
  "Noun_inf": "Word-level grammar",
  "Noun_number": "Word-level grammar",
  "Prepositions": "Word-level grammar",
  "Conjunctions": "Word-level grammar",
  "Adjectives": "Word-level grammar",
  "Comparative_adj": "Word-level grammar",
  "Superlative_adj": "Word-level grammar",
  "Prepositional_adjective": "Word-level grammar",
  "Adj_as_collective": "Word-level grammar",
  "Adverbs": "Word-level grammar",
  "Comparative_adv": "Word-level grammar",
  "Superlative_adv": "Word-level grammar",
  "Prepositional_adv": "Word-level grammar",
  "Numerals": "Word-level grammar",
  "Pronouns": "Word-level grammar",
  "Agreement_errors": "Complex grammar",
  "Word_order": "Complex grammar",
  "Standard": "Complex grammar",
  "Emphatic": "Complex grammar",
  "Cleft": "Complex grammar",
  "Interrogative": "Complex grammar",
  "Abs_comp_clause": "Complex grammar",
  "Exclamation": "Complex grammar",
  "Title_structure": "Complex grammar",
  "Note_structure": "Complex grammar",
  "Conditionals": "Complex grammar",
  "Attributes": "Complex grammar",
  "Relative_clause": "Complex grammar",
  "Defining": "Complex grammar",
  "Non_defining": "Complex grammar",
  "Coordinate": "Complex grammar",
  "Attr_participial": "Complex grammar",
  "Lack_par_constr": "Complex grammar",
  "Negation": "Complex grammar",
  "Comparative_constr": "Complex grammar",
  "Numerical": "Complex grammar",
  "Confusion_of_structures": "Complex grammar",
  "Vocabulary": "Vocabulary",
  "Word_choice": "Vocabulary",
  "lex_item_choice": "Vocabulary",
  "Often_confused": "Vocabulary",
  "lex_part_choice": "Vocabulary",
  "Absence_comp_colloc": "Vocabulary",
  "Redundant": "Vocabulary",
  "Derivation": "Vocabulary",
  "Formational_affixes": "Vocabulary",
  "Suffix": "Vocabulary",
  "Prefix": "Vocabulary",
  "Category_confusion": "Vocabulary",
  "Compound_word": "Vocabulary",
  "Discourse": "Discourse",
  "Ref_device": "Discourse",
  "Coherence": "Discourse",
  "Linking_device": "Discourse",
  "Inappropriate_register": "Discourse",
  "Absence_comp_sent": "Discourse",
  "Redundant_comp": "Discourse",
  "Absence_explanation": "Discourse"
}

Обрабатываем корпус:

In [ ]:
%%time

pd_dict = OrderedDict()

pd_dict["path"] = []
pd_dict["orig_str"] = []
pd_dict["corr_str"] = []
pd_dict["ann_label"] = []
pd_dict["err_start"] = []
pd_dict["err_end"] = []
pd_dict["err_type"] = []


for root, dirs, files in os.walk('data'):
    for file in files:
        if file.endswith(".ann"):
          path = os.path.join(root, file)
          try:
            patch_list = ann_to_patchlist(path, textpatch_format=True)
            for err in patch_list:
              if err.err_type in error_typify:
                pd_dict["path"].append(path)
                pd_dict["orig_str"].append(err.orig_str)
                pd_dict["corr_str"].append(err.corr_str)
                pd_dict["ann_label"].append(err.name)
                pd_dict["err_start"].append(err.start)
                pd_dict["err_end"].append(err.end)
                pd_dict["err_type"].append(err.err_type)
          except Exception as e:
            print("Failed at", file, "with", str(e))

CPU times: user 39.9 s, sys: 533 ms, total: 40.5 s
Wall time: 40.6 s


Как много у нас вхождений ошибок?

In [ ]:
len(pd_dict["orig_str"])

203415

Пора положить данные в `DataFrame`.

In [ ]:
error_df = pd.DataFrame(pd_dict)

error_df.sample(5)

,path,orig_str,corr_str,ann_label,err_start,err_end,err_type
111660,data/exam/Exam2014/ZEv_36_1.ann,traveled,travelled,T1,96,104,Spelling
198258,data/old IELTS/IELTS2016/JSl_34_2.ann,chep,cheap,T2,153,157,Spelling
202170,data/old IELTS/IELTS2016/best_works/40_2.ann,but,", and yet",T5,1876,1879,Linking_device
198745,data/old IELTS/IELTS2016/EKu_67_2.ann,occuping,occupying,T13,1199,1207,Spelling
55401,data/exam/Exam2017/VSa_1-138/VSa_76_2.ann,In to conclude,In conclusion,T8,1299,1313,Category_confusion


Добавим код для извлечения предложений:

In [ ]:
def get_sentence_by_range(start, end, txt, sentencize=sent_tokenize):
  sents = sentencize(txt)
  r_starts, r_ends = [0], [0]
  subst = ""
  for s in sents:
    r_starts.append(txt.find(s, r_ends[-1]))
    r_ends.append(r_starts[-1] + len(s))
    b, e = r_starts[-1], r_ends[-1]
    if b <= end <= e:
      break
  for b, e in zip(r_starts, r_ends):
    if b <= start <= e:
      break
  return txt[b:r_ends[-1]]

def pd_get_context(err_start, err_end, ann_path):
  with open(ann_path[:-3] + "txt", "r", encoding="utf-8") as intext:
    text = intext.read()
  return get_sentence_by_range(err_start, err_end, text)

Эта операция занимает около двух минут, поэтому прикрутим к ней прогрессбар:

In [ ]:
error_df["context"] = error_df.progress_apply(lambda x: pd_get_context(x["err_start"], x["err_end"], x["path"]), axis=1)

Пора проверить – как выглядит наш датасет?

In [ ]:
error_df.sample(5)

,path,orig_str,corr_str,ann_label,err_start,err_end,err_type,context
153141,data/exam/Exam2018/EEr_34_1.ann,", who are overweight",who are overweight,T11,733,753,Defining,"Comparing with the boys, who are overweight, t..."
197292,data/old IELTS/IELTS2016/EKu_5_2.ann,thereputation,the reputation,T9,871,884,Spelling,"Secondly, the chances to adopt to normal life ..."
48807,data/exam/Exam2017/ABl/ABl_36_2.ann,help,a help,T14,269,273,Articles,With help of social networks and web-sites peo...
141086,data/exam/undefined/AKhr_35_2.ann,law-maker's,law-makers',T56,1447,1458,Noun_number,"To conclude with, it is important to understan..."
11446,data/2012-2014/esl_01290.ann,can not,cannot,T2,90,97,Spelling,I can not agree that architecture is the mothe...


Теперь скачаем два списка дубликатов и таблицу с примером вывода.

In [ ]:
!mkdir tables
!wget -q https://storage.googleapis.com/ml-bucket-isikus/practice_dup/for_practice.xlsx -P tables
!wget -q https://storage.googleapis.com/ml-bucket-isikus/practice_dup/rest_of_duplicates.xlsx -P tables
!wget -q https://storage.googleapis.com/ml-bucket-isikus/practice_dup/result_ex_merged.xlsx -P tables

Создадим функции для проверки актуальности пары дубликатов и поиска предпоследнего вхождения:

In [ ]:
def matches(path1, path2):
  try:
    with open(path1, "r") as intxt:
      txt1 = intxt.read()
    with open(path2, "r") as intxt:
      txt2 = intxt.read()
    return txt1 == txt2
  except:
    return False

def find_second_last(text, pattern):
  return text.rfind(pattern, 0, text.rfind(pattern))

Наконец, обработаем списки дублирующихся файлов и пропишем необходимое название файла для файла результата:

In [ ]:
group_1 = pd.read_excel("tables/for_practice.xlsx")
group_1["Exam2018"] = "data/exam/" + group_1["Exam2018"]
group_1["Exam2019"] = "data/exam/" + group_1["Exam2019"]
group_1["Matching"] = group_1.apply(lambda x: matches(x["Exam2018"], x["Exam2019"]), axis=1)
group_1 = group_1.loc[group_1["Matching"]].reset_index(drop=True)
group_1["Result_Name"] = group_1["Exam2019"].apply(lambda x: "Results/Res" + x[x.find("_"):-4])

In [ ]:
group_2 = pd.read_excel("tables/rest_of_duplicates.xlsx")
group_2["Text1"] = "data/exam/" + group_2["Text1"]
group_2["Text2"] = "data/exam/" + group_2["Text2"]
group_2["Matching"] = group_2.apply(lambda x: matches(x["Text1"], x["Text2"]), axis=1)
group_2 = group_2.loc[group_2["Matching"]].reset_index(drop=True)
group_2["Result_Name"] = group_2["Text2"].apply(lambda x: "Results2/Res" + x[find_second_last(x, "_"):-4])

group_2

,Text1,Text2,Matching,Result_Name
0,data/exam/Exam2019/ABu_143_2.txt,data/exam/Exam2019/ABu_215_2.txt,True,Results2/Res_215_2
1,data/exam/Exam2019/ABu_143_1.txt,data/exam/Exam2019/ABu_215_1.txt,True,Results2/Res_215_1
2,data/exam/Exam2015/KT_17_2.txt,data/exam/Test2015/KT_17_2.txt,True,Results2/Res_17_2
3,data/exam/Exam2015/KT_10_2.txt,data/exam/Test2015/KT_10_2.txt,True,Results2/Res_10_2
4,data/exam/Exam2015/KT_10_1.txt,data/exam/Test2015/KT_10_1.txt,True,Results2/Res_10_1
5,data/exam/Exam2015/KT_14_2.txt,data/exam/Test2015/KT_14_2.txt,True,Results2/Res_14_2
6,data/exam/Exam2015/KT_12_2.txt,data/exam/Test2015/KT_12_2.txt,True,Results2/Res_12_2
7,data/exam/Exam2015/KT_19_2.txt,data/exam/Test2015/KT_19_2.txt,True,Results2/Res_19_2
8,data/exam/Exam2015/KT_14_1.txt,data/exam/Test2015/KT_14_1.txt,True,Results2/Res_14_1
9,data/exam/Exam2015/KT_19_1.txt,data/exam/Test2015/KT_19_1.txt,True,Results2/Res_19_1


In [ ]:
output_example = pd.read_excel("tables/result_ex_merged.xlsx")

### Задание

У нас есть датафрейм со всеми записями ошибок в REALEC:

In [ ]:
error_df.sample(5)

,path,orig_str,corr_str,ann_label,err_start,err_end,err_type,context
109347,data/exam/Exam2014/DZu_24_1.ann,write,write down,T9,661,666,Absence_comp_colloc,Other Underground railway system I want to wri...
116377,data/exam/Exam2014/DAr_4_1.ann,eldery,elderly,T16,943,949,Often_confused,In 2050 the picture will change a little: the ...
21686,data/exam/Exam2017_Elizaveta/ekl_10_1.ann,raise,rise,T196,338,343,Often_confused,"Also, in the S. Asia the percentage of unemplo..."
16253,data/Exam_practice/OV201617/151-152/st_17_2.ann,from,among,T12,534,538,Prepositions,Alberta stands out from other provinces.
127539,data/exam/Exam2019/ABu_125_1.ann,sudy,study,T2,240,244,Spelling,"Overall, the vast majority of the visitors bot..."


Также нас есть два списка дублирующихся файлов – `group_1` и `group_2`:

In [ ]:
group_1

,Exam2018,Exam2019,Matching,Result_Name
0,data/exam/Exam2018/EEr_66_1.txt,data/exam/Exam2019/ABu_1_1.txt,True,Results/Res_1_1
1,data/exam/Exam2018/EEr_66_2.txt,data/exam/Exam2019/ABu_1_2.txt,True,Results/Res_1_2
2,data/exam/Exam2018/EEr_5_1.txt,data/exam/Exam2019/ABu_2_1.txt,True,Results/Res_2_1
3,data/exam/Exam2018/EEr_5_2.txt,data/exam/Exam2019/ABu_2_2.txt,True,Results/Res_2_2
4,data/exam/Exam2018/EEr_13_1.txt,data/exam/Exam2019/ABu_3_1.txt,True,Results/Res_3_1
...,...,...,...,...
251,data/exam/Exam2018/EEr_225_1.txt,data/exam/Exam2019/ABu_173_1.txt,True,Results/Res_173_1
252,data/exam/Exam2018/EEr_50_2.txt,data/exam/Exam2019/ABu_28_2.txt,True,Results/Res_28_2
253,data/exam/Exam2018/EEr_17_2.txt,data/exam/Exam2019/ABu_31_2.txt,True,Results/Res_31_2
254,data/exam/Exam2018/EEr_91_1.txt,data/exam/Exam2019/ABu_236_2.txt,True,Results/Res_236_2


In [ ]:
group_2

,Text1,Text2,Matching,Result_Name
0,data/exam/Exam2019/ABu_143_2.txt,data/exam/Exam2019/ABu_215_2.txt,True,Results2/Res_215_2
1,data/exam/Exam2019/ABu_143_1.txt,data/exam/Exam2019/ABu_215_1.txt,True,Results2/Res_215_1
2,data/exam/Exam2015/KT_17_2.txt,data/exam/Test2015/KT_17_2.txt,True,Results2/Res_17_2
3,data/exam/Exam2015/KT_10_2.txt,data/exam/Test2015/KT_10_2.txt,True,Results2/Res_10_2
4,data/exam/Exam2015/KT_10_1.txt,data/exam/Test2015/KT_10_1.txt,True,Results2/Res_10_1
5,data/exam/Exam2015/KT_14_2.txt,data/exam/Test2015/KT_14_2.txt,True,Results2/Res_14_2
6,data/exam/Exam2015/KT_12_2.txt,data/exam/Test2015/KT_12_2.txt,True,Results2/Res_12_2
7,data/exam/Exam2015/KT_19_2.txt,data/exam/Test2015/KT_19_2.txt,True,Results2/Res_19_2
8,data/exam/Exam2015/KT_14_1.txt,data/exam/Test2015/KT_14_1.txt,True,Results2/Res_14_1
9,data/exam/Exam2015/KT_19_1.txt,data/exam/Test2015/KT_19_1.txt,True,Results2/Res_19_1


**Задание**. Необходимо выбрать из общего датафрейма со всеми ошибками те, которые относятся к файлам-дубликатам, и свести нотации ошибок в каждой паре дубликатов согласно следующему алгоритму:

* Если аннотация ошибок из одного файла не пересекается с аннотацией ошибок в другом файле, занести её в соответствующий файл в папке `Results` или `Results2`;
* Если пара аннотаций ошибок в файлах-дубликатах полностью пересекаются и полностью совпадают (у `err1` и `err2` совпадают `err_start`, `err_end`, `err_type` и `corr_str`), также занести любую из идентичных аннотаций в соответствующий файл в папке `Results` или `Results2`;
* Если же области ошибок пересекаются частично, или если они пересекаются полностью, но хотя бы одно из значений `err_start`, `err_end`, `err_type` или `corr_str` у ошибок не совпадает, такие случаи следует занести в отдельный датафрейм расхождения в аннотациях.

Понятие *«аннотации не пересекаются»* означает, что для данной записи `err1` в `file1` нет такой записи `err2` в `file2`, чтобы `range(err2.start, err2.end)` пересекался с аналогичным `range` для записи `err1`. Это можно проверить функцией `range_intersect`:

In [ ]:
def get_name_from_path(st):
  end = 0
  start = 0
  for i in range(len(st)-1,0,-1):
    if st[i]=='.':
      end = i
    if st[i]=='/':
      start = i + 1
      break
  return st[start:end]

In [ ]:
error_df['FileName'] = error_df['path'].apply(get_name_from_path)
group_1['FileName1'] = group_1['Exam2018'].apply(get_name_from_path)
group_1['FileName2'] = group_1['Exam2019'].apply(get_name_from_path)
group_2['FileName1'] = group_2['Text1'].apply(get_name_from_path)
group_2['FileName2'] = group_2['Text2'].apply(get_name_from_path)

In [ ]:
def patchobj_to_brat_ann(patch):
  p = patch
  h = str(hash(repr(p)))
  T = "T" + h[-4:]
  A = "A" + h[-8:-4]
  retstr = "{}\t{} {} {}\t{}\n".format(T, p["err_type"], p["err_start"], p["err_end"], p["orig_str"])
  if patch.corr_str == "":
    retstr += "{}\tDelete {}".format(A, T)
  else:
    retstr += "{}\tAnnotatorNotes {}\t{}".format("#"+T[1:], T, p["corr_str"])
  return retstr

In [ ]:
output = pd.DataFrame(columns=output_example.columns)

In [ ]:
def range_intersect(range1, range2):
  _1 = set(range1)
  _12 = _1.intersection(range2)
  return len(_12) > 0

In [ ]:
!mkdir Results Results2
count=0
for index, row in group_1.iterrows():

  err_1 = error_df.loc[error_df['FileName']==row['FileName1']]
  err_2 = error_df.loc[error_df['FileName']==row['FileName2']]
  res = ""

  for i,r in err_1.iterrows():
    flag = True
    for j,k in err_2.iterrows():
      if range_intersect(range(r.err_start,r.err_end),range(k.err_start,k.err_end)) or (r.err_start==k.err_start and r.err_end==k.err_end and (r.err_type!=k.err_typy or r.corr_str!=k.corr_str)):
        flag=False
    if flag:
      res+=patchobj_to_brat_ann(r)
    else:
       output.loc[count]=[r.path,row['Result_Name'],r.ann_label,range(r.err_start,r.err_end),r.err_type,r.orig_str,r.corr_str,r.context]
       count+=1
  for i,r in err_2.iterrows():
    flag = True
    for j,k in err_1.iterrows():
      if range_intersect(range(r.err_start,r.err_end),range(k.err_start,k.err_end)) or (r.err_start==k.err_start and r.err_end==k.err_end and (r.err_type!=k.err_typy or r.corr_str!=k.corr_str)):
        flag=False
    if flag:
      res+=patchobj_to_brat_ann(r)
    else:
      output.loc[count]=[r.path,row['Result_Name'],r.ann_label,range(r.err_start,r.err_end),r.err_type,r.orig_str,r.corr_str,r.context]
      count+=1
  !cp {row['Exam2018']} {row['Result_Name']+".txt"}
  !echo {"\""+res+"\""} > {row['Result_Name'] + ".ann"}


T2995	Punctuation 467 468	'
#2995	AnnotatorNotes T2995	T9748 Noun_number 198 209 female ones
/bin/bash: line 6: A9046: command not found
T9416	Articles 33 43	the family
#9416	AnnotatorNotes T9416	familyT0125	Superlative_adj 52 59	greater
#0125	AnnotatorNotes T0125	greatestT8484	Articles 184 194	the family
#8484	AnnotatorNotes T8484	familyT7267	Articles 342 352	the family
#7267	AnnotatorNotes T7267	familyT1569	Punctuation 432 435	why
#1569	AnnotatorNotes T1569	whyT1179	Superlative_adj 1023 1030	greater
#1179	AnnotatorNotes T1179	greatestT7962	Inappropriate_register 1117 1125	a lot of
#7962	AnnotatorNotes T7962	manyT4898	Inappropriate_register 1183 1187	kids
#4898	AnnotatorNotes T4898	childrenT3489	Articles 1288 1297	streets
#3489	AnnotatorNotes T3489	the streetT8954 Noun_number 1288 1297 streets
/bin/bash: line 19: A8756: command not found
/bin/bash: line 20: A0527: command not found
/bin/bash: line 22: A0608: command not found
/bin/bash: line 23: A1627: command not found
/bin/bash: lin

In [ ]:
for index, row in group_2.iterrows():

  err_1 = error_df.loc[error_df['FileName']==row['FileName1']]
  err_2 = error_df.loc[error_df['FileName']==row['FileName2']]
  res = ""

  for i,r in err_1.iterrows():
    flag = True
    for j,k in err_2.iterrows():
      if range_intersect(range(r.err_start,r.err_end),range(k.err_start,k.err_end)) or (r.err_start==k.err_start and r.err_end==k.err_end and (r.err_type!=k.err_typy or r.corr_str!=k.corr_str)):
        flag=False
    if flag:
      res+=patchobj_to_brat_ann(r)
    else:
       output.loc[count]=[r.path,row['Result_Name'],r.ann_label,range(r.err_start,r.err_end),r.err_type,r.orig_str,r.corr_str,r.context]
       count+=1
  for i,r in err_2.iterrows():
    flag = True
    for j,k in err_1.iterrows():
      if range_intersect(range(r.err_start,r.err_end),range(k.err_start,k.err_end)) or (r.err_start==k.err_start and r.err_end==k.err_end and (r.err_type!=k.err_typy or r.corr_str!=k.corr_str)):
        flag=False
    if flag:
      res+=patchobj_to_brat_ann(r)
    else:
      output.loc[count]=[r.path,row['Result_Name'],r.ann_label,range(r.err_start,r.err_end),r.err_type,r.orig_str,r.corr_str,r.context]
      count+=1
  !cp {row['Text1']} {row['Result_Name']+".txt"}
  !echo {"\""+res+"\""} > {row['Result_Name'] + ".ann"}


это для очищения папок для тестов

In [ ]:
os.chdir("../")
files = os.listdir()
data = []
for file in files:
    !rm {file}

rm: cannot remove 'run': Is a directory
rm: cannot remove 'var': Is a directory
rm: cannot remove 'mnt': Is a directory
rm: cannot remove 'lib64': Is a directory
rm: cannot remove 'root': Is a directory
rm: cannot remove 'opt': Is a directory
rm: cannot remove 'etc': Is a directory
rm: cannot remove 'bin': Is a directory
rm: cannot remove 'media': Is a directory
rm: cannot remove 'srv': Is a directory
rm: cannot remove 'boot': Is a directory
rm: cannot remove 'dev': Is a directory
rm: cannot remove 'home': Is a directory
rm: cannot remove 'tmp': Is a directory
rm: cannot remove 'proc': Is a directory
rm: cannot remove 'usr': Is a directory
rm: cannot remove 'sys': Is a directory
rm: cannot remove 'sbin': Is a directory
rm: cannot remove 'lib': Is a directory
rm: cannot remove 'content': Is a directory
rm: cannot remove 'tools': Is a directory
rm: cannot remove 'datalab': Is a directory
rm: cannot remove 'swift': Is a directory
rm: cannot remove 'tensorflow-1.15.2': Is a directory
rm: c

In [ ]:
output.to_excel("output.xlsx")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Переводить строки из датафрейма в записи в итоговом файле `.ann` следует с помощью следующей функции:

Пример:

In [ ]:
error_df.loc[9349]

path                           data/exam/Exam2019/ABu_13_1.ann
orig_str                                            the appeal
corr_str                                        the popularity
ann_label                                                  T13
err_start                                                  699
err_end                                                    709
err_type                                           Word_choice
context      Concerning the appeal of regular physical acti...
Name: 9349, dtype: object

In [ ]:
print(patchobj_to_brat_ann(error_df.loc[9349]))

T2427	Word_choice 699 709	the appeal
#2427	AnnotatorNotes T2427	the popularity


Сведённые файлы для дубликатов из `group_1` должны называться согласно значению в столбце `Result_Name` и лежать в папке `Results`; для `group_2`, соответственно, результаты должны лежать в папке `Results2`. В папке должны быть тексты эссе (без изменений) и сведённые файлы аннотаций. Например, для вхождения `Results/Res_1_1` в папке `Results` должен появиться текстовый файл `Res_1_1.txt` с текстом эссе и файл аннотации `Res_1_1.ann` со сведёнными аннотациями.

Отдельный датафрейм расхождения в аннотациях должен выглядеть следующим образом (можно сделать два разных для `group_1` и `group_2`, а можно положить всё в один):

In [ ]:
output_example

,duplicate_path,resulting_path,ann_label,ann_range,err_type,orig_str,corr_str,context
0,data/exam/Exam2019/ABu_4_1.ann,Results/Res_1_1,T4,312 325,Tense_choice,was increased,was increasing,During the whole period the number of children...
1,data/exam/Exam2018/EEr_49_1.ann,Results/Res_1_1,T4,312 325,Tense_form,was increased,was increasing,During the whole period the number of children...
2,data/exam/Exam2019/ABu_4_1.ann,Results/Res_1_1,T6,497 514,Tense_choice,were also changed,was also changing,The amount of children who did different kinds...
3,data/exam/Exam2018/EEr_49_1.ann,Results/Res_1_1,T5,497 514,Voice,were also changed,has also changed,The amount of children who did different kinds...
4,data/exam/Exam2019/ABu_4_1.ann,Results/Res_1_1,T12,497 514,Agreement_errors,were also changed,was also changing,The amount of children who did different kinds...
5,data/exam/Exam2018/EEr_49_1.ann,Results/Res_1_1,T8,497 514,Agreement_errors,were also changed,has also changed,The amount of children who did different kinds...


В качестве результата выполнения задания нужно прислать [мне](https://vk.com/itorubarov) (Ване) ссылки на загруженные на Гугл-диск архивы папок `Results` и `Results2`, а также датафрейм расхождений `output`, сохранённый как таблица Excel, и ссылку на тетрадку с кодом (не забыв открыть ко всему доступ!).

Привожу пример кода архивирования, сохранения таблицы как xlsx-файла и загрузки всего на гугл-диск. Допустим, в папках `Results` и `Results2` всё готово, и собран единый датафрейм расхождений `output` – тогда заархивировать, сохранить и загрузить всё это можно так:

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# архивируем Results
!zip -q -r dedup_results.zip Results

# архивируем Results2
!zip -q -r dedup_results2.zip Results2

# сохраняем датафрейм output в формате Excel
output.to_excel("output.xlsx", index=False)

# загружаем всё на свой гугл-диск
!cp dedup_results.zip /content/gdrive/My\ Drive/dedup_results.zip
!cp dedup_results2.zip /content/gdrive/My\ Drive/dedup_results2.zip
!cp dedup_output.xlsx /content/gdrive/My\ Drive/dedup_output.xlsx


zip error: Nothing to do! (try: zip -q -r dedup_results.zip . -i Results)

zip error: Nothing to do! (try: zip -q -r dedup_results2.zip . -i Results2)
cp: cannot stat 'dedup_results.zip': No such file or directory
cp: cannot stat 'dedup_results2.zip': No such file or directory
cp: cannot stat 'dedup_output.xlsx': No such file or directory
